In [1]:
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from netCDF4 import Dataset
import xarray as xr
from matplotlib.colors import TwoSlopeNorm
from pathlib import Path
import pandas as pd
import h5py

In [2]:
### Read in data and do some minimal processing
parent_dir = Path('/scratch/orybchuk/wakedynamics/bcs-ldm/simulations/072415/large_campaigns/precursor/postprocessing/reformatted')

b_files = list(parent_dir.glob('inflow_b1lev*.nc'))
b_files.sort()
n_campaigns = len(b_files)

In [3]:
### Read in the AMR-Wind config file and grab config inputs
fconfig = Path('/projects/wakedynamics/orybchuk/bcs-ldm/simulations/072415/large_campaigns/precursor/abl_precursor.i')

## Get the names of all the campaigns
sampling_names = []
with open(fconfig, 'r') as f:
    for line in f:
        if 'incflo.post_processing' in line:
            tmpline = line.split()
            for item in tmpline:
                if 'sampling' in item:
                    sampling_names.append(item)
sampling_names.sort()

## Manually specify azimuth and elevation tables
az_table = np.array([195.0, 193.0, 191.0, 189.0, 187.0, 185.0, 183.0, 181.0, 179.0, 177.0, 175.0, 173.0, 171.0, 169.0, 167.0, 165.0])
el_table = np.array([  0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0,   0.0])

In [4]:
### Reformat inflow boundary conditions
for icampaign, fcampaign in enumerate(b_files):
    if icampaign % 25 == 0: print(datetime.now(), icampaign, '...')
    campaign_id = fcampaign.name[12:16]
    assert f'sampling{campaign_id}' in sampling_names, f"sampling{campaign_id} is not in the AMR-Wind config!"

    ### ~~~~~~~~~~ Read b1-level data ~~~~~~~~~~
    ds_b1 = xr.open_dataset(fcampaign)
    
    ### ~~~~~~~~~~ Set up the b2-level coordinates and initialize Dataset ~~~~~~~~~~
    ## Check that the user-input tables match the config file
    with open(fconfig, 'r') as f:
        for line in f:
            # if (f'sampling{campaign_id}' in line) and ('time_table' in line):
            #     tt_line_raw = line.split()
            #     time_table = np.array(tt_line_raw[1+tt_line_raw.index('='):]).astype(float)
            if f'sampling{campaign_id}.inflow-scan.azimuth_table' in line:
                az_line_raw = line.split()
                az_table_raw = np.array(az_line_raw[1+az_line_raw.index('='):]).astype(float)
            if f'sampling{campaign_id}.inflow-scan.elevation_table' in line:
                el_line_raw = line.split()
                el_table_raw = np.array(el_line_raw[1+el_line_raw.index('='):]).astype(float)
    for curr_az in az_table:
        assert curr_az in np.unique(az_table_raw), f"Unexpected azimuth value {curr_az} in the manually specified azimuth table!"
    for curr_el in el_table:
        assert curr_el in np.unique(az_table_raw), f"Unexpected elevation value {curr_el} in the manually specified elevation table!"
            
    ## Beam
    n_unique_beams = len(az_table)
    coordsc_beam = np.arange(n_unique_beams)

    ## Rep
    n_whole_reps = np.floor(len(ds_b1['time'])/n_unique_beams).astype(int)
    n_remainder_reps = len(ds_b1['time']) % n_unique_beams
    # print(f"Total number of whole reps: {n_whole_reps}; Number of extra beam measurements discarded: {n_remainder_reps}")
    coordsc_rep = np.arange(n_whole_reps)

    ## Range
    coordsc_range = ds_b1['range_gate'].values

    ## Initialize Dataset
    coords_b2 = {'beam': coordsc_beam,
                'rep': coordsc_rep,
                'range': coordsc_range}
    ds_b2 = xr.Dataset(coords=coords_b2)
    
    ### ~~~~~~~~~~ Calculate b2-level data variables ~~~~~~~~~~
    ## Time
    timec = np.reshape(ds_b1['time'].values, (n_unique_beams, n_whole_reps), order='F')
    ds_b2['time'] = (('beam', 'rep'), timec)
    assert (ds_b2['time'].isel(beam=0, rep=1).values - ds_b2['time'].isel(beam=0, rep=0).values > \
               ds_b2['time'].isel(beam=0, rep=0).values - ds_b2['time'].isel(beam=1, rep=0).values), \
               "The time of one rep is shorter than moving over one beam? Likely reordering error."

    ## Azimuth and elevation
    ds_b2['azimuth'] = (('beam'), az_table)
    ds_b2['elevation'] = (('beam'), el_table)

    ## X, Y, Z
    range_tiled = np.tile(ds_b2['range'].values, (n_unique_beams,1)).T
    az_tiled = np.tile(ds_b2['azimuth'].values, (len(ds_b2['range']),1))
    el_tiled = np.tile(ds_b2['elevation'].values, (len(ds_b2['range']),1))
    X_tiled = range_tiled * np.cos(np.deg2rad(az_tiled)) * np.cos(np.deg2rad(el_tiled))
    Y_tiled = range_tiled * np.sin(np.deg2rad(az_tiled)) * np.cos(np.deg2rad(el_tiled))
    Z_tiled = range_tiled * np.sin(np.deg2rad(el_tiled))

    ds_b2['X'] = (('range', 'beam'), X_tiled)
    ds_b2['Y'] = (('range', 'beam'), Y_tiled)
    ds_b2['Z'] = (('range', 'beam'), Z_tiled)

    # Radial wind speed
    rwsc = np.nan * np.zeros(((len(ds_b2['range']), n_unique_beams, n_whole_reps)))
    for irange in range(len(ds_b2['range'])):  # Reshape slowly range-by-range because of confusing 3D reshape behavior
        rwsc[irange, :, :] = np.reshape(ds_b1['los_wspd_avg'].isel(range_gate=irange).values, (n_unique_beams, n_whole_reps), order='F').copy()    
    # rwsc = np.reshape(ds_b1['los_wspd_avg'].values, (len(ds_b2['range']), n_unique_beams, n_whole_reps))  # TODO: Is this reshape in the correct order?? Do a visual check.
    ds_b2['rws'] = (('range', 'beam', 'rep'), rwsc)
    
    ds_b2.to_netcdf(Path(parent_dir,f'inflow_b2lev{campaign_id}.nc'))

2024-05-17 15:36:19.199376 0 ...
2024-05-17 15:36:31.625627 25 ...
2024-05-17 15:36:38.229411 50 ...
2024-05-17 15:36:41.769583 75 ...
2024-05-17 15:36:46.348484 100 ...
2024-05-17 15:36:49.989214 125 ...
2024-05-17 15:36:54.113433 150 ...
2024-05-17 15:36:57.338209 175 ...
2024-05-17 15:37:00.998324 200 ...
2024-05-17 15:37:03.973095 225 ...
2024-05-17 15:37:07.385052 250 ...
2024-05-17 15:37:10.702277 275 ...
2024-05-17 15:37:14.234482 300 ...
2024-05-17 15:37:17.854125 325 ...


# Sanity check b2-level data

In [5]:
tmp0 = xr.open_dataset(Path(parent_dir,f'inflow_b2lev0000.nc'))
tmp1 = xr.open_dataset(Path(parent_dir,f'inflow_b2lev0001.nc'))
tmp2 = xr.open_dataset(Path(parent_dir,f'inflow_b2lev0002.nc'))

In [6]:
tmp0

<xarray.Dataset>
Dimensions:    (beam: 16, rep: 200, range: 83)
Coordinates:
  * beam       (beam) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * rep        (rep) int64 0 1 2 3 4 5 6 7 8 ... 192 193 194 195 196 197 198 199
  * range      (range) float64 0.0 12.0 24.0 36.0 ... 948.0 960.0 972.0 984.0
Data variables:
    time       (beam, rep) datetime64[ns] ...
    azimuth    (beam) float64 ...
    elevation  (beam) float64 ...
    X          (range, beam) float64 ...
    Y          (range, beam) float64 ...
    Z          (range, beam) float64 ...
    rws        (range, beam, rep) float64 ...

In [7]:
tmp1

<xarray.Dataset>
Dimensions:    (beam: 16, rep: 200, range: 83)
Coordinates:
  * beam       (beam) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15
  * rep        (rep) int64 0 1 2 3 4 5 6 7 8 ... 192 193 194 195 196 197 198 199
  * range      (range) float64 0.0 12.0 24.0 36.0 ... 948.0 960.0 972.0 984.0
Data variables:
    time       (beam, rep) datetime64[ns] ...
    azimuth    (beam) float64 ...
    elevation  (beam) float64 ...
    X          (range, beam) float64 ...
    Y          (range, beam) float64 ...
    Z          (range, beam) float64 ...
    rws        (range, beam, rep) float64 ...